In [1]:
# %load_ext tensorboard
# Clear any logs from previous runs
# !rm -rf ./logs/

In [2]:
import numpy as np
import pandas as pd
import time, os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.utils import to_categorical
from tensorboard.plugins.hparams import api as hp

from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.models import Model

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split as split

import random
from math import floor
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from scipy.integrate import odeint
import scipy
import pandas as pd
np.random.seed(0)

2022-12-16 16:44:05.823211: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-16 16:44:06.001889: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-16 16:44:06.001927: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-16 16:44:07.126495: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

In [3]:
url = 'https://raw.githubusercontent.com/igomezv/nnogada/main/data/jla.csv'
df=pd.read_csv(url)
N = len(df.values)
randomize = np.random.permutation(N)
data = df.values[randomize]
N = len(df.values)
z = data[:,0] 
y = data[:,1:3] ### coge el resto de variables a predecir 
y[:,1] = y[:,1]**2+data[:,2]
np.shape(y)

(740, 2)

In [4]:
scalerz = StandardScaler()
scalerz.fit(z.reshape(-1,1))
z = scalerz.transform(z.reshape(-1,1))

In [5]:
### Modificar para incluir phanteon como test
split = 0.8
ntrain = int(split * len(z))
indx = [ntrain]
X_train, X_test = np.split(z, indx)
Y_train, Y_test = np.split(y, indx)
np.shape(X_train), np.shape(X_test), Y_train.shape, Y_test.shape

((592, 1), (148, 1), (592, 2), (148, 2))

In [6]:
# loss -> val_loss
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min',
                                   min_delta=0.0,
                                   patience=200,
                                   restore_best_weights=True, verbose=True)
                                   ]

n_cols = 1

In [7]:
HP_BATCHSIZE = hp.HParam('batch_size', hp.Discrete([2, 4, 8, 16]))
HP_LAYERS =    hp.HParam('layers', hp.Discrete([1, 2, 3, 4]))
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([50, 100, 150, 200]))
HP_LEARNING  = hp.HParam('learning_rate', hp.Discrete([1e-4,1e-3]))
# HP_NUM_UNITS3 = hp.HParam('num_units3', hp.Discrete([50, 100, 150, 200]))
# HP_NUM_UNITS4 = hp.HParam('num_units4', hp.Discrete([2, 5, 10]))
# HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.0, 0.2))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'Adadelta']))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam']))

In [8]:
epochs=200

In [9]:
# sess = tf.Session()
# writer = tf.summary.FileWriter("/tmp/tfvgg", sess.graph)
# init = tf.initialize_all_variables()
# sess.run(init)
# with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
#     hp.hparams_config(
#         hparams=[HP_NUM_UNITS1, HP_NUM_UNITS2, HP_NUM_UNITS3, HP_NUM_UNITS4,
#                  HP_OPTIMIZER, HP_BATCHSIZE],
#         metrics=[hp.Metric('loss', display_name="Loss")])

In [10]:
# METRIC_ACCURACY = 'accuracy'
with tf.summary.create_file_writer('logs/hparam_tuning3').as_default():
# with tf.summary.FileWriter('logs/hparam_tuning', sess.graph):
#     init = tf.initialize_all_variables()
#     sess.run(init)
    hp.hparams_config(
        hparams=[HP_LAYERS,
                 HP_NUM_UNITS,
                 HP_LEARNING, 
                 HP_BATCHSIZE],
        metrics=[hp.Metric('loss', display_name="Loss")])

2022-12-16 16:44:08.660772: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-16 16:44:08.661055: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-16 16:44:08.661131: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-12-16 16:44:08.661196: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-12-16 16:44:08.661259: W tensorflow/c

In [11]:
def train_test_model(hparams):    
    
    # Train LSTM model and predict on validation set
    model = tf.keras.Sequential()
    model.add(Dense(int(X_train.shape[1])))
#     model.add(Dense(hparams[HP_NUM_UNITS], input_shape=(int(X_train.shape[1]),)))
    
    for i in range(hparams[HP_LAYERS]):        
        model.add(Dense(hparams[HP_NUM_UNITS], activation='relu'))
    model.add(Dense(2, activation='linear'))
     
    optimizer = tf.keras.optimizers.Adam(learning_rate=hparams[HP_LEARNING], beta_1=0.9, beta_2=0.999, epsilon=1e-3)
    model.compile(
            optimizer=optimizer,
            loss='mse', 
            metrics=['mean_squared_error'])
    
    # Run with 1 epoch to speed things up for demo purposes

    model.fit(X_train, Y_train, epochs=epochs, validation_data=(X_test, Y_test),
              callbacks=callbacks, batch_size=hparams[HP_BATCHSIZE], shuffle=False, verbose=0)

    _, loss = model.evaluate(X_test, Y_test)
    
    return loss

In [12]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        loss = train_test_model(hparams)
        tf.summary.scalar("loss", loss, step=1)
        return loss

In [13]:
session_num = 0
datos = []

for deep_layers in HP_LAYERS.domain.values:
    for num_units in HP_NUM_UNITS.domain.values:
        for learning_rate in HP_LEARNING.domain.values:
            for batch_size in HP_BATCHSIZE.domain.values:
                t = time.time()
                hparams = {

                    HP_LAYERS: deep_layers,
                    HP_NUM_UNITS: num_units,
                    HP_LEARNING: learning_rate,
                    HP_BATCHSIZE: batch_size,
                }
                run_name = "run-%d" % session_num
                print('\n--- Starting trial: %s' % run_name)
                print({h.name: hparams[h] for h in hparams})
                score = run('logs/hparam_tuning3/' + run_name, hparams)
                t = time.time()-t
                session_num += 1
                print("Loss:", score, "Tiempo transcurrido:", t)
            
            datos.append([deep_layers, num_units, learning_rate, batch_size, score, t])

print(session_num)


--- Starting trial: run-0
{'layers': 1, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 2}
5/5 [==============================] - 0s 3ms/step - loss: 0.0642 - mean_squared_error: 0.0642
Loss: 0.06418228149414062 Tiempo transcurrido: 99.28778219223022

--- Starting trial: run-1
{'layers': 1, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 4}
5/5 [==============================] - 0s 2ms/step - loss: 0.5545 - mean_squared_error: 0.5545
Loss: 0.55450040102005 Tiempo transcurrido: 53.26859164237976

--- Starting trial: run-2
{'layers': 1, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 8}
5/5 [==============================] - 0s 2ms/step - loss: 0.2227 - mean_squared_error: 0.2227
Loss: 0.22272852063179016 Tiempo transcurrido: 29.13958477973938

--- Starting trial: run-3
{'layers': 1, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 16}
5/5 [==============================] - 0s 2ms/step - loss: 15.2891 - mean_squared_error: 15.2891
Loss: 15.289128303527832 

5/5 [==============================] - 0s 3ms/step - loss: 0.0360 - mean_squared_error: 0.0360
Loss: 0.03599366173148155 Tiempo transcurrido: 20.431771516799927

--- Starting trial: run-32
{'layers': 2, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 2}
5/5 [==============================] - 0s 2ms/step - loss: 0.0333 - mean_squared_error: 0.0333
Loss: 0.033332034945487976 Tiempo transcurrido: 109.27167677879333

--- Starting trial: run-33
{'layers': 2, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 4}
5/5 [==============================] - 0s 2ms/step - loss: 0.0362 - mean_squared_error: 0.0362
Loss: 0.03620372340083122 Tiempo transcurrido: 54.53160357475281

--- Starting trial: run-34
{'layers': 2, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 8}
5/5 [==============================] - 0s 2ms/step - loss: 0.0439 - mean_squared_error: 0.0439
Loss: 0.04391714185476303 Tiempo transcurrido: 34.5291531085968

--- Starting trial: run-35
{'layers': 2, 'num_units': 

5/5 [==============================] - 0s 1ms/step - loss: 0.0320 - mean_squared_error: 0.0320
Loss: 0.03196781873703003 Tiempo transcurrido: 20.089050769805908

--- Starting trial: run-63
{'layers': 2, 'num_units': 200, 'learning_rate': 0.001, 'batch_size': 16}
5/5 [==============================] - 0s 1ms/step - loss: 0.0636 - mean_squared_error: 0.0636
Loss: 0.06358816474676132 Tiempo transcurrido: 13.619547367095947

--- Starting trial: run-64
{'layers': 3, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 2}
5/5 [==============================] - 0s 1ms/step - loss: 0.0305 - mean_squared_error: 0.0305
Loss: 0.030505811795592308 Tiempo transcurrido: 58.22625136375427

--- Starting trial: run-65
{'layers': 3, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 4}
5/5 [==============================] - 0s 1ms/step - loss: 0.0348 - mean_squared_error: 0.0348
Loss: 0.03478826582431793 Tiempo transcurrido: 31.85734224319458

--- Starting trial: run-66
{'layers': 3, 'num_units'

5/5 [==============================] - 0s 1ms/step - loss: 0.0321 - mean_squared_error: 0.0321
Loss: 0.03206883743405342 Tiempo transcurrido: 37.22696495056152

--- Starting trial: run-94
{'layers': 3, 'num_units': 200, 'learning_rate': 0.001, 'batch_size': 8}
5/5 [==============================] - 0s 1ms/step - loss: 0.0474 - mean_squared_error: 0.0474
Loss: 0.047382909804582596 Tiempo transcurrido: 22.253122091293335

--- Starting trial: run-95
{'layers': 3, 'num_units': 200, 'learning_rate': 0.001, 'batch_size': 16}
5/5 [==============================] - 0s 1ms/step - loss: 0.0366 - mean_squared_error: 0.0366
Loss: 0.036567699164152145 Tiempo transcurrido: 14.295861721038818

--- Starting trial: run-96
{'layers': 4, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 2}
5/5 [==============================] - 0s 1ms/step - loss: 0.0396 - mean_squared_error: 0.0396
Loss: 0.03963076323270798 Tiempo transcurrido: 60.9944429397583

--- Starting trial: run-97
{'layers': 4, 'num_units'

5/5 [==============================] - 0s 1ms/step - loss: 0.0661 - mean_squared_error: 0.0661
Loss: 0.06605222076177597 Tiempo transcurrido: 82.40811944007874

--- Starting trial: run-125
{'layers': 4, 'num_units': 200, 'learning_rate': 0.001, 'batch_size': 4}
5/5 [==============================] - 0s 1ms/step - loss: 0.0356 - mean_squared_error: 0.0356
Loss: 0.03559416905045509 Tiempo transcurrido: 36.58378219604492

--- Starting trial: run-126
{'layers': 4, 'num_units': 200, 'learning_rate': 0.001, 'batch_size': 8}
5/5 [==============================] - 0s 1ms/step - loss: 0.0347 - mean_squared_error: 0.0347
Loss: 0.03468960523605347 Tiempo transcurrido: 21.500879049301147

--- Starting trial: run-127
{'layers': 4, 'num_units': 200, 'learning_rate': 0.001, 'batch_size': 16}
5/5 [==============================] - 0s 1ms/step - loss: 0.0727 - mean_squared_error: 0.0727
Loss: 0.07269438356161118 Tiempo transcurrido: 13.365513563156128
128


In [14]:
filename = "historial_jla_tunning.txt"
df = pd.DataFrame(datos, columns = ["Deep size", "Num units", "Learning rate", "Batch size", "MSE", "Tiempo de ejecución"])

df.sort_values(by=["MSE", "Tiempo de ejecución"], ascending=[True, True], ignore_index=True, inplace=True)

df.to_csv(filename, header=True, index=False, sep='\t', mode='w') # a=append, w=overwrite

In [15]:
df.head(2)

,Deep size,Num units,Learning rate,Batch size,MSE,Tiempo de ejecución
0,4,100,0.0010,16,0.032425,14.040920
1,4,200,0.0001,16,0.034097,17.128865


In [16]:
total_time = float(np.sum(df[["Tiempo de ejecución"]])/60)

print("Elapsed time {:.3f} minutes".format(total_time))

Elapsed time 9.306 minutes


In [17]:
df["Tiempo de ejecución"]

0     14.040920
1     17.128865
2     20.431772
3     20.677273
4     14.295862
5     15.666862
6     13.879282
7     13.213167
8     21.171222
9     14.487042
10    18.075716
11    15.194508
12    13.265509
13    18.085516
14    19.771043
15    19.766472
16    17.406092
17    18.145130
18    13.689334
19    12.644456
20    15.322505
21    42.351656
22    12.668020
23    13.619547
24    13.261324
25    13.365514
26    22.588393
27    14.129901
28    21.326003
29    22.334208
30    19.770683
31    16.584038
Name: Tiempo de ejecución, dtype: float64